
# Table of Contents



I began working through the [Project Euler](https://projecteuler.net/) problems in Python, using `org-mode`
to organize my work, `timeit` to benchmark solutions, and HackerRank to provide
additional challenges. Here, I've began translating my solutions to Julia, using
the `@btime` macro from `BenchmarkTools` to compare run times.

Initialize Julia session with imports.



In [1]:
using BenchmarkTools
using DelimitedFiles
using Test
@btime 0;

0.016 ns (0 allocations: 0 bytes)

## 1 Multiples of 3 and 5



If we list all the natural numbers below 10 that are multiples of 3 or 5,
we get 3, 5, 6 and 9. The sum of these multiples is 23.

Find the sum of all the multiples of 3 or 5 below 1000.



### A:



We can write the solution in terms of triangular numbers, making sure to
subtract all multiples of fifteen since they were double-counted by the first
two terms:

\begin{eqnarray*}
S & = & (3+6+\ldots+999) + (5+10+\ldots+995) - (15+30+\ldots+990) \\
  & = & 3(1+2+\ldots+333) + 5(1+2+\ldots+199) - 15(1+2+\ldots+66) \\
  & = & 3T_{333} + 5T_{199} - 15T_{66}
\end{eqnarray*}

It is quicker to solve it analytically, even if generality is lost. The
triangular number $T_N$ can be found in $O(1)$. Check out the Wikipedia
entry for [triangular numbers](https://en.wikipedia.org/wiki/Triangular_number) if you aren't already familiar with this formula.

$$ T_N = 1+2+\ldots+N = \frac{N(N+1)}{2}. $$



In [1]:
"Find the ``n``th triangular number."
triangular(n::T) where {T<:Integer} = convert(T,div(n*(n+1),2))

@test triangular(1) == 1
@test triangular(2) == 3
@test triangular(100) == 5050

Test Passed

The $T_N$ we use for a factor, $f$, is easily found given our upper bound,
$n$. We know $fN\leq n$ and $N$ must be an integer. Therefore,

$$ N = \lfloor n/f \rfloor $$

The sum of all multiples of $f_1,f_2$ that are less than $n$ is written as
follows:

$$ S_n(f_1,f_2) = f_1\frac{N_1(N_1+1)}{2} + f_2\frac{N_2(N_2+1)}{2} -
f_3\frac{N_3(N_3+1)}{2} $$

where $N_i = \lfloor n/f_i \rfloor$ and $f_3 = f_1f_2$.



In [1]:
"Find the sum of multiples of ``3`` and ``5`` less than or equal to ``n``."
pe001(n::T) where {T<:Integer} = convert(T,3*triangular(div(n,3))+5*triangular(div(n,5))-15*triangular(div(n,15)))

@test pe001(10-1) == 23

lim = 100
@btime pe001(lim-1)

29.129 ns (1 allocation: 16 bytes)
2318

There is a way to generalize this for list of factors, but that is beyond the
scope of this problem.



## 2 Even Fibonacci numbers



Each new term in the Fibonacci sequence is generated by adding the previous
two terms. By starting with 1 and 2, the first 10 terms will be:
1, 2, 3, 5, 8, 13, 21, 34, 55, 89, &#x2026;

By considering the terms in the Fibonacci sequence whose values do not exceed
four million, find the sum of the even-valued terms.



### A:



Briefly ignoring that the problem statement contains the wrong starting numbers
for the Fibonacci sequence, this is quick and easy to solve by brute force since
there aren't too many Fibonacci numbers below 4 million. To be as efficient as
possible, I'll preallocate the array. To figure out the length of the array, we
need to know the index of the greatest Fibonacci number that does not exceed 4
million. Consider the formula for the $n$th Fibonacci number.

$$ F_n = \frac{\varphi^{n+1}-(-\varphi)^{-(n+1)}}{\sqrt{5}} $$

where $\varphi=\frac{1+\sqrt{5}}{2}$. There's a useful simplification we can
make if we look closely at the second term and substitute
$-\varphi^{-1}=-0.618$.

\begin{eqnarray*}
F_n & = & \frac{\varphi^{n+1}}{\sqrt{5}} - \frac{(-\varphi^{-1})^{n+1}}{\sqrt{5}} \\
    & = & \frac{\varphi^{n+1}}{\sqrt{5}} - \frac{(-0.618)^{n+1}}{\sqrt{5}}
\end{eqnarray*}

Since $ \left| \frac{(-0.618)^{n+1}}{\sqrt{5}} \right| < \frac{1}{2} $ for all
$n\geq0$, we can eliminate the second term and round to the nearest integer
with either the nearest integer function or the ceiling function.

\begin{eqnarray*}
F_n & = & \left[ \frac{\varphi^{n+1}}{\sqrt{5}} \right] \\
    & = & \lceil \frac{\varphi^{n+1}}{\sqrt{5}} - \frac{1}{2} \rceil
\end{eqnarray*}

If we want a number not exceeding an upper bound $\beta$, then
$F_n\leq\beta$.

\begin{eqnarray*}
\frac{\varphi^{n+1}}{\sqrt{5}}
                 - \frac{1}{2} & \leq & \beta \\
                 \varphi^{n+1} & \leq & \sqrt{5}\cdot\left( \beta + \frac{1}{2} \right) \\
               (n+1)\ln\varphi & \leq & \frac{1}{2}\ln5+\ln\left( \beta + \frac{1}{2} \right) \\
                           n+1 & \leq & \frac{\frac{1}{2}\ln5+\ln\left( \beta + \frac{1}{2} \right)}
                                             {\ln\varphi} \\
                           n+1 & =    & \lfloor \frac{\frac{1}{2}\ln5 + \ln\left( \beta + \frac{1}{2} \right)}
                                                     {\ln\varphi} \rfloor
\end{eqnarray*}

NOTE: Because the starting sequence is shifted by one index, I've left every
expression in terms of $n+1$. If the starting sequence were the regular
$1,1$, I'd just replace all the $n+1$ with $n$. Thus, the following
function is expressed in terms of $n$.



In [1]:
"Find the index of the Fibonacci not exceeding ``limit``."
function fibonacci_below(limit::T) where {T<:Integer}
    return trunc(T, (log(5)/2+log(limit+0.5)) / log((1+sqrt(5))/2))
end

@test fibonacci_below(1) == 2
@test fibonacci_below(2) == 3
@test fibonacci_below(3) == 4
@test fibonacci_below(4) == 4
@test fibonacci_below(5) == 5

Test Passed

Again, the starting sequence is shifted by an index and doesn't introduce
additional even numbers, so I'm treating this as if the correct sequence was
provided.



In [1]:
"Generate a sequence of Fibonacci numbers, ``F_n<=limit``."
function fibonacci(limit::T) where {T<:Integer}
    n = fibonacci_below(limit)
    F = zeros(T,n)
    F[1:2] = [1,1]
    for i in 3:n
        F[i] = F[i-1] + F[i-2]
    end
    return F
end

@test fibonacci(2) == [1,1,2]
@test fibonacci(89) == [1,1,2,3,5,8,13,21,34,55,89]

Test Passed

Some marginal savings can be made by recognizing that only $F_{3i+2}$ are even
and using list slicing to extract those numbers quickly, rather than checking if
every number is even. See the table below as clarification for why this pattern
arises.

$$ \text{odd} + \text{even} = \text{odd} $$

$$ \text{odd} + \text{odd} = \text{even} $$

| i|1|2|3|4|5|6|7|8|&#x2026;|3i|3i+1|3i+2|
|---|---|---|---|---|---|---|---|---|---|---|---|---|
| $parity(F_i)$|odd|even|odd|odd|even|odd|odd|even|&#x2026;|odd|odd|even|



In [1]:
lim = 4*(10^6) - 1
@btime sum(fibonacci(lim)[3:3:end])

198.152 ns (5 allocations: 688 bytes)
4613732

## 3 Largest prime factor



The prime factors of 13195 are 5, 7, 13 and 29.

What is the largest prime factor of the number 600851475143?



### A:



This can be solved by prime factorization, keeping in mind that all primes
above two are odd and indivisible by three. In the code below we don't bother to
save the exponents since we only really care about the factors. Additionally, we
save a huge amount of time by breaking the loop as soon as the placeholder $n$
is guaranteed to be prime or 1.



In [1]:
"Find the prime factors of ``n``. Returns only primes, not their exponents."
function prime_factors(n::T) where {T<:Integer}
    p = T[]
    if n%2 == 0
        push!(p,2)
        while n%2 == 0
            n = div(n,2)
        end
    end

    f = 3
    while f*f <= n
        if n%f == 0
            push!(p,f)
            while n%f == 0
                n = div(n,f)
            end
        end
        f += (f%6 > 1 ? 2 : 4)
    end

    if n > 1
        push!(p,n)
    end
    return p
end

@test prime_factors(1) == []
@test prime_factors(2) == [2]
@test prime_factors(4) == [2]
@test prime_factors(6) == [2,3]
@test prime_factors(24) == [2,3]
@test prime_factors(25) == [5]

Test Passed

For large numbers and no supplied list of primes the run time of the algorithm
is $O(n)$.



In [1]:
num = 600851475143
@btime prime_factors(num)[end]

3.448 μs (3 allocations: 144 bytes)
6857

## 4 Largest palindrome product



A palindromic number reads the same both ways. The largest palindrome made
from the product of two 2-digit numbers is 9009 = 91 × 99.

Find the largest palindrome made from the product of two 3-digit numbers.



### A:



HackerRank asks for the largest palindrome product below a given limit, rather
than that which is a product of two $k$ digit numbers. The following is my
HackerRank solution, rather than an extensible Project Euler solution.

I build a list of palindrome products then find the largest one below a certain
limit. First, a check for palindromes.



In [1]:
"Determine if a number ``n`` is a palindrome."
ispalindrome(n::String) = n == reverse(n)
function ispalindrome(n::Integer, b::Integer=10)
    n_str = string(n,base=b)
    return n_str == reverse(n_str)
end

@test !ispalindrome(12)
@test ispalindrome(121)

Test Passed

Then, for each possible palindrome, we check for integer products and terminate
once they are found.



In [1]:
pals = Int[]
for a = 999:-1:100, b = a:-1:100
    p = a*b
    if ispalindrome(p)
        push!(pals,p)
    end
end

maximum(pals)

906609

## 5 Smallest multiple



2520 is the smallest number that can be divided by each of the numbers from
1 to 10 without any remainder.

What is the smallest positive number that is evenly divisible by all of the
numbers from 1 to 20?



### A:



Consider these two composite numbers and their forms in prime factorization.

$$ 12 = 2^2\cdot3^1\cdot5^0 \\
   15 = 2^0\cdot3^1\cdot5^1 $$

We know that $lcm(12,15)=60=2^2\cdot3^1\cdot5^1$. In other words, the prime
factorization gives us a way to quickly find the LCM. In general terms this
means

\begin{eqnarray*}
k_a          & = & \prod_i p_i^{a_i} \\
k_b          & = & \prod_i p_i^{b_i} \\
lcm(k_a,k_b) & = & \prod_i p_i^{\max\{a_i,b_i\}}
\end{eqnarray*}

For all $k\leq n\in\mathbb{N}$, the LCM must be the product of the primes
raised to their maximum possible power within the limit given. I used [the Sieve
of Eratosthenes](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes) to get a list of primes below an upper bound. You should check
out the Wiki page for a cool visualization of the algorithm. Compared to my
Python implementation, this Julia implementation is must faster. I'm impressed.



In [1]:
"Find all primes less than ``n`` with the Sieve of Eratosthenes."
function esieve(n::T) where {T<:Integer}
    if n <= 2
        return T[]
    end

    n_odd = div(n,2) - 1
    isprime = trues(n_odd)
    for k in 3:2:trunc(T,sqrt(n))
        if isprime[div(k,2)]
            for i in div(k*k,2):k:n_odd
                isprime[i] = false
            end
        end
    end

    primes = zeros(T,sum(isprime)+1)
    primes[1] = 2
    count = 2
    for i in 1:n_odd
        if isprime[i]
            primes[count] = 2*i+1
            count += 1
        end
    end

    return primes
end

@test esieve(2) == []
@test esieve(3) == [2]
@test esieve(10) == [2,3,5,7]
@test esieve(20) == [2,3,5,7,11,13,17,19]

@btime esieve(10^6);

1.901 ms (5 allocations: 674.56 KiB)

For the sequence we were given, every natural number up to a limit $n$ is
represented. For every prime $p\leq n$, there is a maximum possible exponent
$a$ such that

$$ p^a \leq n $$

Solving for $a$ gives

$$ a = \lfloor \log_p n \rfloor = \lfloor \ln n / \ln p \rfloor $$

Therefore, the least common multiple of the series $1,2,...,n$ is the product

$$ lcm(1,2,\ldots,n) = p_1^{a_1}p_2^{a_2}\ldots p_k^{a_k} $$

where $p_k\leq n$ are prime and $a_k = \lfloor \log_p n \rfloor$.



In [1]:
"Find the smallest common multiple of the sequence of natural numbers `i<=n'."
function lcm_naturals(n::T) where {T<:Integer}
    if n == 1
        return T(1)
    else
        return convert(T, reduce(*, [p ^ trunc(log(n) / log(p)) for p in esieve(n+1)]))
    end
end

@test lcm_naturals(10) == 2520

@btime lcm_naturals(20)

327.004 ns (5 allocations: 448 bytes)
232792560

## 6 Sum square difference



The sum of the squares of the first ten natural numbers is,

$$ 1^2 + 2^2 + \ldots + 10^2 = 385 $$

The square of the sum of the first ten natural numbers is,

$$ (1 + 2 + \ldots + 10)^2 = 55^2 = 3025 $$

Hence the difference between the sum of the squares of the first ten natural
numbers and the square of the sum is 3025 − 385 = 2640.

Find the difference between the square of the sum and the sum of the squares of
the first one hundred natural numbers.



### A:



Recall from problem 1 the formula for the $n$th triangular number, $T_n$.

$$ T_n = 1+2+\ldots+n = \frac{n(n+1)}{2} $$

Now consider the [square pyramidal numbers](https://en.wikipedia.org/wiki/Square_pyramidal_number), $P_n$. Again, if you aren't
familiar with this series or its derivation, check out the Wikipedia page.

$$ P_n = 1^2+2^2+\ldots+n^2 = \frac{n(n+1)(2n+1)}{6} $$



In [1]:
"Find the ``n``th square pyramidal number."
squarepyrimidal(n::T) where {T<:Integer} = convert(T, div(n*(n+1)*(2n+1),6))

@test squarepyrimidal(1) == 1
@test squarepyrimidal(2) == 5
@test squarepyrimidal(4) == 30

Test Passed

Let $f$ be the difference between the sum of the squares and the square of the
sum for the first $n$ natural numbers.

\begin{eqnarray*}
f(n) & = & \left[\sum_{i=1}^n i \right]^2 -
           \left[\sum_{i=1}^n i^2 \right] \\
     & = & T_n^2 - P_n
\end{eqnarray*}



In [1]:
"""
For the first ``n`` natural numbers, find the difference between the square of
the sum and the sum of the squares.
"""
pe006(n::Integer) = triangular(n)^2 - squarepyrimidal(n)

@test pe006(10) == 2640

@btime pe006(100)

0.016 ns (0 allocations: 0 bytes)
25164150

## 7 10001st prime



By listing the first six prime numbers: 2, 3, 5, 7, 11, and 13, we can see
that the 6th prime is 13.

What is the 10 001st prime number?



### A:



We should use the Sieve of Eratosthenes (see problem 5) to find this, but first
we need to approximate what our upper bound should be. [According to Dusart](https://en.wikipedia.org/wiki/Prime_number_theorem#Approximations_for_the_nth_prime_number), the
$n$th prime, $p_n$, is bound by the following inequality.

$$ n(\log n + \log\log n - 1) < p_n < n(\log n + \log\log n) $$

for $n\geq6$.



In [1]:
"Find the lower and upper bounds of the ``n``th prime."
function prime_bounds(n::T) where {T<:Integer}
    if n < 6
        return T[1,14]
    else
        lim = log(n)+log(log(n))
        return [trunc(T,n*(lim-1)),trunc(T,n*lim+1)]
    end
end

@test prime_bounds(4) == [1,14]
@test prime_bounds(6) == [8,15]
@test prime_bounds(10) == [21,32]

Test Passed

I used the upper bound from the above calculation to perform a Sieve of
Eratosthenes and return the 10001st prime. It is interesting to note how
close the prime is to the lower bound.

The overall algorithm should be about the same complexity as the sieve,
$O(n\log\log n)$. Any additional complexity results from systematic
overestimation of the upper bound.



In [1]:
i = 10001
bounds = @btime prime_bounds(i)
println(bounds)
@btime esieve(bounds[2])[i]

62.777 ns (1 allocation: 96 bytes)
[104318, 114320]
  220.718 μs (6 allocations: 91.83 KiB)
104743

## 8 Largest product in a series



The four adjacent digits in the 1000-digit number that have the greatest
product are 9 × 9 × 8 × 9 = 5832.



In [1]:
number = """
73167176531330624919225119674426574742355349194934
96983520312774506326239578318016984801869478851843
85861560789112949495459501737958331952853208805511
12540698747158523863050715693290963295227443043557
66896648950445244523161731856403098711121722383113
62229893423380308135336276614282806444486645238749
30358907296290491560440772390713810515859307960866
70172427121883998797908792274921901699720888093776
65727333001053367881220235421809751254540594752243
52584907711670556013604839586446706324415722155397
53697817977846174064955149290862569321978468622482
83972241375657056057490261407972968652414535100474
82166370484403199890008895243450658541227588666881
16427171479924442928230863465674813919123162824586
17866458359124566529476545682848912883142607690042
24219022671055626321111109370544217506941658960408
07198403850962455444362981230987879927244284909188
84580156166097919133875499200524063689912560717606
05886116467109405077541002256983155200055935729725
71636269561882670428252483600823257530420752963450
"""

tmp = reduce(*, readdlm(IOBuffer(number),String))
v = Int[parse(Int, d) for d in tmp];

Find the thirteen adjacent digits in the 1000-digit number that have the
greatest product. What is the value of this product?



### A:



This is a moving window problem. Let $N$ be a number with $n$ digits,  $\mathbf{p}$
be a vector where the elements $p_i$ are the products of $w > 1$ adjacent
digits of $N$, starting with $d_i$.

$$ N = d_1d_2...d_n $$

$$ \mathbf{p} \in \mathbb{R}^{n-w+1} $$

$$ p_i = \prod_{k=i}^{i+w-1} d_k $$



In [1]:
"Find the products of ``w`` adjacent elements in the vector ``v``."
function pe008(v::Array{T}, w::T) where {T<:Integer}
     n = length(v)
     p = ones(T, n-w+1)
     for i in 1:w
         p .*= v[i:n+i-w]
     end
     return p
end

@test maximum(pe008(v,4)) == 5832

@btime maximum(pe008(v,13))

8.121 μs (15 allocations: 110.27 KiB)
23514624000

## 9 Special Pythagorean triplet



A Pythagorean triple is a set of three natural numbers, $a < b < c$, for
which,

$$ a^2 + b^2 = c^2 $$

For example, $3^2 + 4^2 = 9 + 16 = 25 = 5^2$.

There exists exactly one Pythagorean triplet for which $a + b + c = 1000$.
Find the product $abc$.



### A:



Using [Euclid's formula](https://en.wikipedia.org/wiki/Pythagorean_triple#Generating_a_triple) for the primitive Pythagorean triples:

\begin{eqnarray*}
a & = & m^2-n^2 \\
b & = & 2mn \\
c & = & m^2+n^2
\end{eqnarray*}

where $m>n>0$, $m$ and $n$ are coprime ($gcd(m,n)=1$) and one is even.
Since we don't necessarily need these triples to be primitive, we'll rewrite the
formula by adding in some common divisor, $d$.

\begin{eqnarray*}
a & = & d(m^2-n^2) \\
b & = & d(2mn) \\
c & = & d(m^2+n^2)
\end{eqnarray*}

Introducing a constraint on the sum, $a+b+c=s$, gives

\begin{eqnarray*}
s & = & a + b + c \\
  & = & d(m^2-n^2) + d(2mn) + d(m^2+n^2) \\
  & = & 2dm^2 + 2dmn \\
  & = & 2dm(m+n) \\
  & = & 2dmk
\end{eqnarray*}

A few interesting things to point out here. Try plugging in numbers if any of
these points don't make sense.

1.  $s$ must be divisible by 2.
2.  $s/2$ must be divisible by $m$.
3.  $m < \sqrt{s/2}$ because $m < k = m+n$.
4.  $s/(2m)$ must be divisible by $k = m+n$.
5.  $m$ and $k = m+n$ are coprime ($gcd(m,k)=1$) since $m$ and $n$ are coprime.
6.  $k = m+n$ is odd.

From these observations we can write an algorithm that, in the outer loop,
checks all valid $m$, and, in the inner loop, checks all valid $k$ for the
chosen $m$. We know that there is only one triple for $a+b+c=1000$, but if
we spend the extra time to check all possible entries, we can generalize this
code for problem 39 and 75.



In [1]:
"Find the greatest common divisor of two integers."
function gcd(a::T, b::T) where {T<:Integer}
    while b != 0
        (a, b) = (b, a%b)
    end
    return a
end

@test gcd(4,5) == 1
@test gcd(4,10) == 2

"""
Find all Pythagorean triples ``a^2+b^2=c^2`` such that ``a+b+c=s`` and
``0<a<b<c<s``.
"""
function pythagorean_triples_from_sum(s::T) where {T<:Integer}
    if s%2 == 1
        return Array{Array{T}}[]
    end
    trip = Array{T}[]
    for m in 2:trunc(T,sqrt(s/2))
        if (s/2)%m == 0
            k = (m%2==0 ? m+1 : m+2)
            while (k < 2*m) && k <= s/(2*m)
                if (s/(2*m))%k==0 && gcd(m,k) == 1
                    d = s/(2*k*m)
                    n = k - m
                    a = d*(m^2-n^2)
                    b = d*2*m*n
                    c = d*(m^2+n^2)
                    push!(trip,T[min(a,b),max(a,b),c])
                end
                k += 2
            end
        end
    end
    return trip
end

@test pythagorean_triples_from_sum(12)[1] == [3,4,5]
@test pythagorean_triples_from_sum(40)[1] == [8,15,17]

trip = @btime pythagorean_triples_from_sum(1000)
println(trip[1])
println(reduce(*,trip[1]))

384.083 ns (3 allocations: 240 bytes)
[200, 375, 425]
31875000

The algorithm is extremely efficient for this small sum and is also scalable to
large sums since it only needs to check the primitive triples.



In [1]:
@btime pythagorean_triples_from_sum(10^9)

1.613 ms (5 allocations: 464 bytes)
3-element Array{Array{Int64,N} where N,1}:
 [200000000, 375000000, 425000000]
 [218750000, 360000000, 421250000]
 [23437500, 488000000, 488562500]

## 10 Summation of primes



The sum of the primes below 10 is 2 + 3 + 5 + 7 = 17.

Find the sum of all the primes below two million.



### A:



Sieve of Eratosthenes.



In [1]:
@btime sum(esieve(2000000))

3.832 ms (5 allocations: 1.26 MiB)
142913828922